In [ ]:
import os
import pandas as pd

CHEMIN_DOSSIER_DONNÉES = "../../../data_test/diamant"
DATE_FICHIER = '2022_09_01'
encoding = 'utf-8'

In [ ]:
ann_ms_tdp_et = pd.read_csv(os.path.join(CHEMIN_DOSSIER_DONNÉES, f'ANN_MS_TDP_ET_{DATE_FICHIER}.CSV'), delimiter=";", encoding=encoding, dtype='str')

In [ ]:
colonnes = [
    "Nombre de CDD de remplacement",
    "Taux de prestations externes sur les prestations directes",
    "Taux d'ETP vacants au 31/12",
    "Taux de rotation du personnel sur effectifs réels",
    "Taux d'absentéisme (hors formation)",
    "Taux d'absentéisme pour maladie ordinaire/courte durée",
    "Taux d'absentéisme pour maladie moyenne durée",
    "Taux d'absentéisme pour maladie longue durée",
    "Taux d'absentéisme pour maternité/paternité",
    "Taux d'absentéisme pour accident du travail / maladie professionnelle",
    "Taux d'absentéisme pour congés spéciaux dont sans solde"
]
index = [
    "Finess",
    "Année"
]

## Doublons

In [ ]:
a_plus_d_une_donnée_par_finess_par_année = (ann_ms_tdp_et.groupby(index).nunique() > 1).any(axis=1)

## Valeurs manquantes

In [ ]:
ann_ms_tdp_et[colonnes].count() / 

In [ ]:
(~ann_ms_tdp_et[[
    "Taux d'absentéisme (hors formation)",
    "Taux d'absentéisme pour maladie ordinaire/courte durée",
    "Taux d'absentéisme pour maladie moyenne durée",
    "Taux d'absentéisme pour maladie longue durée",
    "Taux d'absentéisme pour maternité/paternité",
    "Taux d'absentéisme pour accident du travail / maladie professionnelle",
    "Taux d'absentéisme pour congés spéciaux dont sans solde"
]].isna().apply(any, axis=1)).sum()

In [ ]:
ann_ms_tdp_et[colonnes].isna().sum() / ann_ms_tdp_et.shape[0]

## Années renseignées

In [ ]:
ann_ms_tdp_et['Année'].unique()

In [ ]:
ann_ms_tdp_et['Année'].hist()

In [ ]:
ann_ms_tdp_et.groupby('Finess').count()['Année'].hist()

## Cohérence des données

### Présence de taux négatifs

In [ ]:
for colonne in colonnes:
    print(colonne, " ===> ", ann_ms_tdp_et[ann_ms_tdp_et[colonne].astype(float) < 0].shape[0])

In [ ]:
ann_ms_tdp_et[ann_ms_tdp_et["Taux d'ETP vacants au 31/12"].astype(float) < 0][colonnes]

###  Présence de taux > 100 %

In [ ]:
for colonne in colonnes:
    print(colonne, " ===> ", ann_ms_tdp_et[ann_ms_tdp_et[colonne].astype(float) > 1].shape[0])

In [ ]:
ann_ms_tdp_et[ann_ms_tdp_et["Taux de rotation du personnel sur effectifs réels"].astype(float) > 1][colonnes]

## Cohérence des taux d'absentéisme

In [ ]:
comparaison_des_sommes = ann_ms_tdp_et[["Taux d'absentéisme (hors formation)"]].astype(float).round(decimals=2)

In [ ]:
comparaison_des_sommes['somme_des_taux'] = ann_ms_tdp_et[[
    "Taux d'absentéisme pour maladie ordinaire/courte durée",
    "Taux d'absentéisme pour maladie moyenne durée",
    "Taux d'absentéisme pour maladie longue durée",
    "Taux d'absentéisme pour maternité/paternité",
    "Taux d'absentéisme pour accident du travail / maladie professionnelle",
    "Taux d'absentéisme pour congés spéciaux dont sans solde"
]].astype(float).sum(axis=1, skipna=False).round(decimals=2)

In [ ]:
somme_des_taux_absentéisme_incohérent = comparaison_des_sommes["Taux d'absentéisme (hors formation)"] != comparaison_des_sommes["somme_des_taux"]

In [ ]:
comparaison_des_sommes[somme_des_taux_absentéisme_incohérent]

### 0 pour Taux d'absentéisme (hors formation) & NaN pour la somme des autres taux

In [ ]:
taux_d_absentéisme_à_0_et_somme_à_nan = (comparaison_des_sommes["Taux d'absentéisme (hors formation)"] == 0) & (comparaison_des_sommes["somme_des_taux"].isna())

In [ ]:
comparaison_des_sommes[taux_d_absentéisme_à_0_et_somme_à_nan].shape[0] / ann_ms_tdp_et.shape[0]

Quand les taux d'absentéismes ne sont pas remplis (NAN) leur somme (Taux d'absentéisme (hors formation)) est à 0 => est-ce que c'est DIAMANT qui le calcule ? 

### NaN pour Taux d'absentéisme (hors formation) & NaN pour la somme des autres taux

In [ ]:
taux_d_absentéisme_à_nan_et_somme_à_nan = (comparaison_des_sommes["Taux d'absentéisme (hors formation)"].isna())

In [ ]:
comparaison_des_sommes[taux_d_absentéisme_à_nan_et_somme_à_nan].shape[0] / ann_ms_tdp_et.shape[0]

In [ ]:
comparaison_des_sommes[(~(taux_d_absentéisme_à_0_et_somme_à_nan | taux_d_absentéisme_à_nan_et_somme_à_nan)) & somme_des_taux_absentéisme_incohérent]

Parfois, la somme des taux n'est pas parfaitement égales au taux d'absentéisme hors formations => erreur d'arrondi par DIAMANT ? Peut-on nous même faire la somme ?

In [ ]:
comparaison_des_sommes.apply(lambda x: abs(x["Taux d'absentéisme (hors formation)"] - x["somme_des_taux"]) > 0.01, axis=1).sum()

In [ ]:
ann_ms_tdp_et[colonnes].astype(float).describe()

In [ ]:
ann_ms_tdp_et[colonnes].astype(float).hist(bins=100, figsize=(10, 20) ,layout=(11, 1))